# Houston Police Dept crime data archive


## Fetch Data
http://www.houstontx.gov/police/cs/crime-stats-archives.htm

URL Format: http://www.houstontx.gov/police/cs/xls/jul16.xls

In [1]:
import pandas as pd
import calendar

monthmap = {k: v.lower() for k,v in enumerate(calendar.month_abbr)}

def month_year_iter( start_month, start_year, end_month, end_year ):
    ym_start= 12*start_year + start_month - 1
    ym_end= 12*end_year + end_month - 1
    for ym in range( ym_start, ym_end ):
        y, m = divmod( ym, 12 )
        yield y, m+1
        
crimes_df = pd.DataFrame()

for year, month in month_year_iter(6, 9, 1, 17):
    url = 'http://www.houstontx.gov/police/cs/xls/'+(monthmap[month] + str(year).zfill(2))+'.xls'
    
    for attempt in range(10):
        try:
            print('Fetching ' + url + '. Attempt ' + str(attempt+1) + '... ', end='')
            df = pd.read_excel(url)
            print('Succeeded')
        except:
            print('Failed')
        else:
            break
    else:
        print('SKIPPED '+ url)

    cols = []
    for col in df.columns:
        col = col.replace(' ', '')
        cols.append(col)
        
    df.columns = cols

    crimes_df = crimes_df.append(df, ignore_index=True)

Fetching http://www.houstontx.gov/police/cs/xls/jun09.xls. Attempt 1... Succeeded
Fetching http://www.houstontx.gov/police/cs/xls/jul09.xls. Attempt 1... WARNING *** file size (3085901) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
Succeeded
Fetching http://www.houstontx.gov/police/cs/xls/aug09.xls. Attempt 1... Succeeded
Fetching http://www.houstontx.gov/police/cs/xls/sep09.xls. Attempt 1... WARNING *** file size (2971767) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
Succeeded
Fetching http://www.houstontx.gov/police/cs/xls/oct09.xls. Attempt 1... Succeeded
Fetching http://www.houstontx.gov/police/cs/xls/nov09.xls. Attempt 1... WARNING *** file size (2845765) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
Succeeded
Fetching http://www.houstontx.gov/police/cs/xls/dec09.xls. Attempt 1... S

# Clean up data

In [4]:
crimes_df.columns

Index(['#Of', '#OfOffenses', '#Offenses', '#offenses', 'Beat', 'BlockRange',
       'Date', 'Field11', 'Hour', 'OffenseType', 'Premise', 'StreetName',
       'Suffix', 'Type', 'Unnamed:1'],
      dtype='object')

In [20]:
crimes_df['Offenses'] = crimes_df['#Offenses'].combine_first(crimes_df['#OfOffenses']).combine_first(crimes_df['#Of']).combine_first(crimes_df['#offenses'])

In [25]:
del crimes_df['#offenses']
del crimes_df['#Of']
del crimes_df['#Offenses']
del crimes_df['#OfOffenses']

In [31]:
del crimes_df['Field11']

In [33]:
del crimes_df['Unnamed:1']

In [34]:
crimes_df.head(2)

,Beat,BlockRange,Date,Hour,OffenseType,Premise,StreetName,Suffix,Type,Offenses
0,18F40,2900-2999,2009-06-24,14,Murder,18A,CROSSVIEW,-,DR,1.0
1,16E20,4800-4899,2009-06-24,10,Murder,20R,MACRIDGE,-,BLVD,1.0


In [35]:
crimes_df.to_csv(r'C:\xc\Houston-crimes.csv')

In [37]:
crimes_df.describe()

,Offenses
count,974480.000000
mean,1.014538
std,0.157721
min,1.000000
25%,1.000000
50%,1.000000
75%,1.000000
max,33.000000


In [42]:
crimes_df

,Beat,BlockRange,Date,Hour,OffenseType,Premise,StreetName,Suffix,Type,Offenses
0,18F40,2900-2999,2009-06-24,14,Murder,18A,CROSSVIEW,,DR,1.0
1,16E20,4800-4899,2009-06-24,10,Murder,20R,MACRIDGE,,BLVD,1.0
2,17E10,6700-6799,2009-06-23,21,Murder,18T,HILLCROFT,,,1.0
3,3B40,900-999,2009-06-08,0,Murder,11R,40TH 1/2,E,ST,1.0
4,9C30,3900-3999,2009-06-03,23,Murder,20R,MANITOU,,DR,1.0
5,18F50,7600-7699,2009-03-29,2,Murder,18N,DASHWOOD,,DR,1.0
6,18F50,5800-5899,2009-06-28,5,Murder,20A,FONDREN,,RD,1.0
7,6B10,11000-11099,2009-06-14,18,Murder,13R,HARDY,W,RD,1.0
8,19G50,10500-10599,2009-05-31,10,Murder,19V,ROCKLEY,,RD,1.0
9,17E30,8800-8899,2009-06-20,2,Murder,20A,BRAESWOOD,S,BLVD,1.0


In [44]:
crimes_df = crimes_df.replace('-', '')
crimes_df['DayOfWeek'] = crimes_df.Date.dt.weekday_name

In [59]:
crimes_df['Address'] = crimes_df.BlockRange.map(str) + ' ' + \
                       crimes_df.Suffix.map(str) + ' ' + \
                       crimes_df.StreetName.map(str) + ' ' + \
                       crimes_df.Type.map(str) + ', Houston, TX'

In [60]:
crimes_df

,Beat,BlockRange,Date,Hour,OffenseType,Premise,StreetName,Suffix,Type,Offenses,DayOfWeek,Address
0,18F40,2900-2999,2009-06-24,14,Murder,18A,CROSSVIEW,,DR,1.0,Wednesday,"2900-2999 CROSSVIEW DR, Houston, TX"
1,16E20,4800-4899,2009-06-24,10,Murder,20R,MACRIDGE,,BLVD,1.0,Wednesday,"4800-4899 MACRIDGE BLVD, Houston, TX"
2,17E10,6700-6799,2009-06-23,21,Murder,18T,HILLCROFT,,,1.0,Tuesday,"6700-6799 HILLCROFT , Houston, TX"
3,3B40,900-999,2009-06-08,0,Murder,11R,40TH 1/2,E,ST,1.0,Monday,"900-999 E 40TH 1/2 ST, Houston, TX"
4,9C30,3900-3999,2009-06-03,23,Murder,20R,MANITOU,,DR,1.0,Wednesday,"3900-3999 MANITOU DR, Houston, TX"
5,18F50,7600-7699,2009-03-29,2,Murder,18N,DASHWOOD,,DR,1.0,Sunday,"7600-7699 DASHWOOD DR, Houston, TX"
6,18F50,5800-5899,2009-06-28,5,Murder,20A,FONDREN,,RD,1.0,Sunday,"5800-5899 FONDREN RD, Houston, TX"
7,6B10,11000-11099,2009-06-14,18,Murder,13R,HARDY,W,RD,1.0,Sunday,"11000-11099 W HARDY RD, Houston, TX"
8,19G50,10500-10599,2009-05-31,10,Murder,19V,ROCKLEY,,RD,1.0,Sunday,"10500-10599 ROCKLEY RD, Houston, TX"
9,17E30,8800-8899,2009-06-20,2,Murder,20A,BRAESWOOD,S,BLVD,1.0,Saturday,"8800-8899 S BRAESWOOD BLVD, Houston, TX"


### Remove UNK from BlockRange

This will improve geocoding

# Geocode Addresses

In [62]:
import pandas as pd
import arcpy

from arcgis.gis import GIS
from arcgis.geocoding import get_geocoders, batch_geocode

gis = GIS("https://geosaurus.maps.arcgis.com/", "rsingh_geosaurus", "P@ssw0rd@123")

In [64]:
df = crimes_df.copy()
batch_size = get_geocoders(gis)[0].properties.locatorProperties.MaxBatchSize
N = len(df)

In [66]:
import arcpy

geoms = []
for i in range(0, N, batch_size):
    start = i
    stop = i+batch_size if i+batch_size < N else N
    print('Geocoding from ' + str(start) + ' to ' + str(stop))
    res = batch_geocode(list(df[start:stop]['Address']))
    
    for index in range(len(res)):
        address = df.ix[start+index,'Address']
       
        try:
            loc = res[index]['location']
            x = loc['x']
            y = loc['y']
            df.ix[start+index,'x'] = x
            df.ix[start+index,'y'] = y
            geoms.append(arcpy.PointGeometry(arcpy.Point(x,y)))
        except:
            print('Unable to geocode Address: ' + address)
            df.ix[start+index,'x'] = None
            df.ix[start+index,'y'] = None
            geoms.append(None)

Geocoding from 0 to 1000
Geocoding from 1000 to 2000
Geocoding from 2000 to 3000
Geocoding from 3000 to 4000
Geocoding from 4000 to 5000
Geocoding from 5000 to 6000
Geocoding from 6000 to 7000
Geocoding from 7000 to 8000
Geocoding from 8000 to 9000
Geocoding from 9000 to 10000
Geocoding from 10000 to 11000
Geocoding from 11000 to 12000
Geocoding from 12000 to 13000
Geocoding from 13000 to 14000
Geocoding from 14000 to 15000
Geocoding from 15000 to 16000
Geocoding from 16000 to 17000
Geocoding from 17000 to 18000
Geocoding from 18000 to 19000
Geocoding from 19000 to 20000
Geocoding from 20000 to 21000
Geocoding from 21000 to 22000
Geocoding from 22000 to 23000
Geocoding from 23000 to 24000
Geocoding from 24000 to 25000
Geocoding from 25000 to 26000
Geocoding from 26000 to 27000
Geocoding from 27000 to 28000
Geocoding from 28000 to 29000
Geocoding from 29000 to 30000
Geocoding from 30000 to 31000
Geocoding from 31000 to 32000
Geocoding from 32000 to 33000
Geocoding from 33000 to 34000
Ge

ConnectionAbortedError: [WinError 10053] An established connection was aborted by the software in your host machine

Since my geocoding ran out of credits, I had to fill the remaining geometries using None...

In [69]:
len(geoms)

947000

In [70]:
N

974484

In [72]:
for j in range (len(geoms), N):
    geoms.append(None)

In [73]:
len(geoms)

974484

## Save out as file geodatabase

In [74]:
from arcgis import SpatialDataFrame as sdf
from arcgis import GeoSeries as gs

In [75]:
from arcpy import Geometry

In [76]:
spdf = sdf(df, geometry=geoms)

In [78]:
spdf.head(2)

,Beat,BlockRange,Date,Hour,OffenseType,Premise,StreetName,Suffix,Type,Offenses,DayOfWeek,Address,x,y,SHAPE
0,18F40,2900-2999,2009-06-24,14,Murder,18A,CROSSVIEW,,DR,1.0,Wednesday,"2900-2999 CROSSVIEW DR, Houston, TX",-95.519717,29.733960,(-95.5197165269997 29.7339604650005 NaN NaN)
1,16E20,4800-4899,2009-06-24,10,Murder,20R,MACRIDGE,,BLVD,1.0,Wednesday,"4800-4899 MACRIDGE BLVD, Houston, TX",-95.451212,29.603143,(-95.4512118389997 29.6031425160004 NaN NaN)


In [96]:
spdf.ix[0, 'SHAPE'] = arcpy.Point(X=-95.5197165269997, Y=29.7339604650005)

In [79]:
spdf.sr = arcpy.SpatialReference(4326)

In [106]:
spdf_copy.dtypes

Beat                  object
BlockRange            object
Date          datetime64[ns]
Hour                  object
OffenseTyp            object
Premise               object
StreetName            object
Suffix                object
Type                  object
Offenses             float64
DayOfWeek             object
Address               object
x                    float64
y                    float64
SHAPE                 object
dtype: object

## Clean up data

Hour column contains strings (with leading quotes), floats and int as well as empty strings. Convert them all to ints

In [132]:
spdf_copy['Hour'].unique()

array([14.0, 10.0, 21.0, 0.0, 23.0, 2.0, 5.0, 18.0, 20.0, 8.0, 6.0, 4.0,
       1.0, 19.0, 22.0, 3.0, 7.0, 11.0, 17.0, 12.0, 9.0, 15.0, 16.0, 13.0,
       '', '07', '05', '22', '15', '10', '19', '09', '08', '14', "'15",
       '02', '12', "'07", '16', "'19", "'18", '21', '17', '11', '13', '18',
       '06', "'00", "'05", "'13", "'14", "'22", "'23", '04', '03', '23',
       '20', '00', "'20", "'16", '01', "'17", "'06", "'03", "'01", "'09",
       "'02", "'10", "'11", "'12", "'21", "'08", "'04", 24], dtype=object)

In [135]:
spdf_copy.loc[spdf_copy['Hour'] == '', 'Hour'] = 24

In [141]:
spdf_copy['Hour'] = spdf_copy['Hour'].apply(lambda x : int(x.replace("'", "")) if type(x) == str else int(x))

In [216]:
spdf_copy['Hour'].unique()

array([14, 10, 21,  0, 23,  2,  5, 18, 20,  8,  6,  4,  1, 19, 22,  3,  7,
       11, 17, 12,  9, 15, 16, 13, 24], dtype=int64)

Code doesnt yet handle None geometries - fix that

In [175]:
spdf_copy['SHAPE'] = spdf_copy['SHAPE'].apply(lambda x : arcpy.PointGeometry(arcpy.Point(0, 0)) if x == '' else x)

In [176]:
spdf_copy.dtypes

Beat                  object
BlockRange            object
Date          datetime64[ns]
Hour                   int64
OffenseTyp            object
Premise               object
StreetName            object
Suffix                object
Type                  object
Offenses             float64
DayOfWeek             object
Address               object
x                    float64
y                    float64
SHAPE                 object
dtype: object

# Export to File Geodatabase

In [214]:
spdf_copy.to_featureclass(out_location=r"C:\xc\Presentations\Demos\Houston\houston.gdb", out_name="houston_crime")

'C:\\xc\\Presentations\\Demos\\Houston\\houston.gdb\\houston_crime'

In [218]:
item = gis.content.add({}, r'C:/xc/Presentations/Demos/Houston/houston.gdb.zip')

In [219]:
item

<Item title:"houston.gdb" type:File Geodatabase owner:rsingh_geosaurus>

In [ ]:
houston_crimes = item.publish()

# Learn from data

In [233]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import log_loss
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import LogisticRegression
import numpy as np

In [269]:
df = spdf_copy

In [270]:
df.head()

,Beat,BlockRange,Date,Hour,OffenseTyp,Premise,StreetName,Suffix,Type,Offenses,DayOfWeek,Address,x,y,SHAPE
0,18F40,2900-2999,2009-06-24,14,Murder,18A,CROSSVIEW,,DR,1.0,Wednesday,"2900-2999 CROSSVIEW DR, Houston, TX",-95.519717,29.733960,(-95.4512118389997 29.6031425160004 NaN NaN)
1,16E20,4800-4899,2009-06-24,10,Murder,20R,MACRIDGE,,BLVD,1.0,Wednesday,"4800-4899 MACRIDGE BLVD, Houston, TX",-95.451212,29.603143,(-95.4512118389997 29.6031425160004 NaN NaN)
2,17E10,6700-6799,2009-06-23,21,Murder,18T,HILLCROFT,,,1.0,Tuesday,"6700-6799 HILLCROFT , Houston, TX",-95.493237,29.707182,(-95.4932365569996 29.7071817070005 NaN NaN)
3,3B40,900-999,2009-06-08,0,Murder,11R,40TH 1/2,E,ST,1.0,Monday,"900-999 E 40TH 1/2 ST, Houston, TX",-95.387272,29.822993,(-95.3872719499997 29.8229928610004 NaN NaN)
4,9C30,3900-3999,2009-06-03,23,Murder,20R,MANITOU,,DR,1.0,Wednesday,"3900-3999 MANITOU DR, Houston, TX",-95.265635,29.799019,(-95.2656352009997 29.7990187260004 NaN NaN)


In [271]:
len(df.StreetName.unique())

12631

In [272]:
len(df.Beat.unique())

239

In [273]:
len(df)

974484

# Remove rows with incomplete data

In [274]:
df = df[df.OffenseTyp != 1]

In [275]:
df = df[df.OffenseTyp != '']

In [279]:
df = df[df.DayOfWeek != '']

In [285]:
len(df)

974464

## Clean up independent variable - Offence Type

In [280]:
df.OffenseTyp.unique()

array(['Murder', 'Rape', 'Robbery', 'Aggravated Assault', 'Burglary',
       'Auto Theft', 'Theft'], dtype=object)

# Preprocess

Vectorize data

In [282]:
offence_encoder = preprocessing.LabelEncoder()
offences = offence_encoder.fit_transform(df['OffenseTyp'])
offences

array([3, 3, 3, ..., 1, 5, 6], dtype=int64)

In [352]:
offence_encoder.classes_

array(['Aggravated Assault', 'Auto Theft', 'Burglary', 'Murder', 'Rape',
       'Robbery', 'Theft'], dtype=object)

# Hour

In [286]:
hour = pd.get_dummies(df.Hour, prefix="hour")
hour.head()

,hour_0,hour_1,hour_2,hour_3,hour_4,hour_5,hour_6,hour_7,hour_8,hour_9,...,hour_15,hour_16,hour_17,hour_18,hour_19,hour_20,hour_21,hour_22,hour_23,hour_24
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [287]:
day = pd.get_dummies(df.DayOfWeek, prefix="day")
day.head()

,day_Friday,day_Monday,day_Saturday,day_Sunday,day_Thursday,day_Tuesday,day_Wednesday
0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,1
2,0,0,0,0,0,1,0
3,0,1,0,0,0,0,0
4,0,0,0,0,0,0,1


In [288]:
beats = pd.get_dummies(df.Beat, prefix="beats")
beats.head()

,beats_,beats_'10H10,beats_'10H20,beats_'10H30,beats_'10H40,beats_'10H50,beats_'10H60,beats_'10H70,beats_'10H80,beats_'11H10,...,beats_8C50,beats_8C60,beats_9C10,beats_9C20,beats_9C30,beats_9C40,beats_HCSO,beats_OOJ,beats_UH-3P,beats_UNK
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [ ]:
days.head()

In [326]:
data = pd.concat([hour, day, beats], axis=1)

In [327]:
data['Offence'] = offences

In [328]:
data.head()

,hour_0,hour_1,hour_2,hour_3,hour_4,hour_5,hour_6,hour_7,hour_8,hour_9,...,beats_8C60,beats_9C10,beats_9C20,beats_9C30,beats_9C40,beats_HCSO,beats_OOJ,beats_UH-3P,beats_UNK,Offence
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,3


In [329]:
training, validation = train_test_split(data, train_size=.75)

In [331]:
features = list(training.columns)[:-1]

# Naïve Bayes 

In [341]:
model_nb = BernoulliNB()
model_nb.fit(training[features], training.Offence)
predicted_nb = np.array(model_nb.predict_proba(validation[features]))
log_loss(validation.Offence, predicted_nb) 

1.2337042371409042

In [355]:
model_nb.score(training[features], training.Offence)

0.54950276938569986

In [356]:
model_nb.score(validation[features], validation.Offence)

0.54892125311966378

# Logistic Regression

In [342]:
model_lr = LogisticRegression(C=.01)
model_lr.fit(training[features], training.Offence)
predicted_lr = np.array(model_lr.predict_proba(validation[features]))
log_loss(validation.Offence, predicted_lr) 

1.2359294601644186

In [357]:
model_lr.score(training[features], training.Offence)

0.54999945269057315

In [358]:
model_lr.score(validation[features], validation.Offence)

0.54921680021016683

In [360]:
model_nb.predict(validation[features][2:3])

array([6], dtype=int64)

In [361]:
validation.Offence[2:3]

234369    6
Name: Offence, dtype: int64

In [362]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
knn.fit(training[features], training.Offence)
predicted = np.array(knn.predict_proba(validation[features]))
log_loss(validation.Offence, predicted)


KeyboardInterrupt



# Support Vector Classifier

In [365]:
data = pd.concat([hour, day, beats], axis=1)

In [366]:
data['Offence'] = offences

In [367]:
data = data[:1000]

In [368]:
training, validation = train_test_split(data, train_size=.75)

In [369]:
features = list(training.columns)[:-1]

In [370]:
from sklearn.svm import SVC

svc = SVC()
svc.fit(training[features], training.Offence)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [371]:
svc.score(validation[features], validation.Offence)

0.93200000000000005

In [372]:
svc.score(training[features], training.Offence)

0.90266666666666662

In [373]:
svc.predict(validation[features][20:23])

array([5, 5, 5], dtype=int64)

In [374]:
validation.Offence[20:23]

605    5
980    5
533    5
Name: Offence, dtype: int64